In [ ]:
# dataset preparation
from starry.score_connection.data import *


file = open('./test/test.json', 'rb')
data = loadConnectionSet(file)

#seqs, matrixH, masks = exampleToTensors(data['connections'][0], 0x100, 0x200)
#print(seqs, matrixH, masks)

examples = list(map(lambda ex: exampleToTensors(ex, 0x100, 0x200), data['connections']))
dataset = batchizeTensorExamples(examples, 4)

print('dataset:', dataset[0]['seq_id'].shape, dataset[0]['seq_position'].shape, dataset[0]['mask'].shape, dataset[0]['matrixH'].shape)
#print('seq_id.0', dataset[0]['seq_id'][0])
#print('seq_position.0', dataset[0]['seq_position'][0][1])


In [ ]:
# sequence masking
import torch
from starry.transformer.models import get_subsequent_mask, get_pad_mask


seq = torch.tensor([[3,2,1], [4,5,6]])
mask1 = get_pad_mask(seq, 1)
mask2 = get_subsequent_mask(seq)
print(mask1)
print(mask2)
print(mask1 & mask2)


In [ ]:
# model test
from os.path import expanduser
import dill as pickle
from starry.score_connection.models import *
from starry.score_connection.data import Dataset


data = pickle.load(open(expanduser('~/data/score/connections/packages/chopin10-3x100.pkl'), 'rb'))
test, = Dataset.loadPackage(data, batch_size=1, splits='0/100')

model = TransformJointerLoss()
batch = next(iter(test))
#pred = model(batch['seq_id'], batch['seq_position'], batch['mask'])
#print(pred)

loss, accuracy = model(batch)
print('loss:', loss)
print('accuracy:', accuracy)


In [ ]:
# matmul test
import torch


t1 = torch.tensor([[1,2,3], [1,2,3]])
t2 = torch.tensor([[4,5,6], [4,5,6]])

t1 = t1.unsqueeze(-2)#.repeat(2, 2, 1, 1)
t2 = t2.unsqueeze(-1)#.repeat(2, 2, 1, 1)
print(t1, t2)

p = t1.matmul(t2).flatten()
print(p)


In [ ]:
# repeat test
import torch


t = torch.tensor([[1,2,3], [1,2,3], [1,2,3]])
r1 = t.repeat(2, 1, 1)
r2 = t.unsqueeze(1).repeat(1, 2, 1)

print(r1)
print(r2)


In [ ]:
# preprocessDataset test
from starry.score_connection.data import *


datasets = preprocessDataset('~/data/score/connections/chopin10-3x100', splits=('0/20',), batch_size=4)
print('datasets:', len(datasets[0]), datasets[0][0])

#import dill as pickle
#pickle.dump(datasets[0], open('test.pkl', 'wb'))


In [ ]:
# dataset loading
import dill as pickle
from starry.score_connection.data import Dataset


data = pickle.load(open('~/data/score/connections/packages/chopin10-3x100.pkl', 'rb'))
training, validation = Dataset.loadPackage(data, batch_size=4, splits='*1,2/100;0/100')

print('training len:', len(training))
print('validation len:', len(validation))

for batch in training:
	print('batch:', batch)
	break


In [ ]:
# prediction
import torch
from os.path import expanduser
import dill as pickle
from starry.score_connection.models import *
from starry.score_connection.data import Dataset


checkpoint = torch.load('./output/model_01.chkpt', map_location='cpu')
model = TransformJointer()
model.load_state_dict(checkpoint['model'])

data = pickle.load(open(expanduser('~/data/score/connections/packages/chopin10-3x100.pkl'), 'rb'))
test, = Dataset.loadPackage(data, batch_size=1, splits='0/100')

batch = next(iter(test))
#print('batch:', batch)

pred = model(batch['seq_id'], batch['seq_position'], batch['mask'])
matrixH = batch['matrixH']

print('target:', matrixH)
print('pred:', pred)
